In [2]:
import pandas as pd
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import os
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = 'SET_YOUR_KEY'

In [3]:
MODEL = 'gpt-3.5-turbo'
TEMP = 0.7
llm = ChatOpenAI(model=MODEL, temperature=TEMP)

In [4]:
def read_dataframe(path = "data.csv"):
    df = pd.read_csv(path)
    return df

In [5]:
df = read_dataframe()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Serial Number     998 non-null    int64  
 1   List Year         998 non-null    int64  
 2   Date Recorded     998 non-null    object 
 3   Town              998 non-null    object 
 4   Address           998 non-null    object 
 5   Assessed Value    998 non-null    int64  
 6   Sale Amount       998 non-null    int64  
 7   Sales Ratio       998 non-null    float64
 8   Property Type     998 non-null    object 
 9   Residential Type  893 non-null    object 
 10  Non Use Code      245 non-null    object 
 11  Assessor Remarks  219 non-null    object 
 12  OPM remarks       31 non-null     object 
 13  Location          998 non-null    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 109.3+ KB


In [6]:
df.head()

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,210002,2021,10/1/2021,Stonington,62 MOSS ST,136500,295000,0.462700,Residential,Single Family,NaN,NaN,NaN,POINT (-71.837214991 41.370295012)
1,210020,2021,10/27/2021,North Branford,2255 FOXON RD,115300,107822,1.069355,Residential,Single Family,14 - Foreclosure,NaN,NaN,POINT (-72.751903 41.338795988)
2,210821,2021,5/16/2022,New Haven,355 NEWHALL ST,132790,295000,0.450100,Residential,Three Family,NaN,NaN,NaN,POINT (-72.92691697 41.33352201)
3,21248,2021,9/16/2022,Oxford,30 OLD COUNTRY ROAD,313000,590000,0.530500,Residential,Single Family,NaN,NaN,NaN,POINT (-73.149303027 41.390997014)
4,2100975,2021,2/23/2022,Stamford,75 HENRY STREET,435930,1600000,0.272456,Apartments,NaN,25 - Other,ENTERPRISE ZONE,NaN,POINT (-73.541341989 41.044746)


In [12]:
def create_save_sql_db(df, db_name='real_estate_sales', table_name='data'):
    """
    Creates and saves a SQLite database from a DataFrame.

    Parameters:
    - df (DataFrame): The DataFrame containing the data to be saved into the database.
    - db_name (str): The name of the SQLite database. Default is 'real_estate_sales'.
    - table_name (str): The name of the table to be created in the database. Default is 'data'.

    Returns:
    - str: The URI (Uniform Resource Identifier) of the saved SQLite database.
           This URI can be used to connect to the database using SQLAlchemy.
    """
    import sqlite3

    conn = sqlite3.connect(f'{db_name}.db')

    df.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()

    return f"sqlite:///{db_name}.db"
uri = create_save_sql_db(df)
uri

'sqlite:///real_estate_sales.db'

CSV file successfully converted to SQLite database. DB Name: real_estate_sales - Table Name: data


In [13]:
def interact_with_database(uri):
    """
    Interact with an SQLite database: print the dialect, usable table names, and run a sample query.

    Parameters:
    uri (str): The URI of the SQLite database.

    Returns:
    None
    """
    # Create an SQLDatabase object from the given URI
    db = SQLDatabase.from_uri(uri)
    return db


In [14]:
db = interact_with_database(uri)

In [ ]:
question = "Compare are the average sales amount in Portland and Orange when it comes to Residential plots"
main_question = {"question": question}


In [199]:
sql_query_chain = create_sql_query_chain(llm, db)
response = sql_query_chain.invoke(main_question)
response

'SELECT "Town", AVG("Sale Amount") AS "Average Sale Amount"\nFROM data\nWHERE "Town" IN (\'Portland\', \'Orange\') AND "Property Type" = \'Residential\'\nGROUP BY "Town"'

In [200]:
execute_query = QuerySQLDataBaseTool(db=db)
query_responce_chain = sql_query_chain | execute_query
query_responce_chain.invoke(main_question)


"[('Orange', 492044.44444444444), ('Portland', 278007.22222222225)]"

In [196]:

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, context, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer_chain = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=sql_query_chain).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_chain
)



In [198]:
question = "Compare are the average sales amount in Portland and Orange when it comes to Residential plots"
main_question = {"question": question}
chain.invoke(main_question)


'The average sale amount for Residential plots in Orange is $492,044.44, while the average sale amount for Residential plots in Portland is $278,007.22. Orange has a higher average sale amount compared to Portland for Residential plots.'